In [ ]:
import numpy as np
import pandas as pd
from sympy import *
import math
import os

In [2]:
#Input Pressure and Temperature values
P = float(input("Enter pressure in psia:")) # psia
T = float(input("Enter Temperature in Farenheit: ")) # Farenheit

Enter pressure in psia: 750
Enter Temperature in Farenheit:  -70


In [3]:
#Import Components Properties
component_properties = pd.read_excel("../ASSIGNMENTS/components.xlsx")
component_properties

,component,composition,Pci,Tci,wi,Mwi,Vci
0,N2,0.0200,493.1,227.49,0.040,28.013,0.0510
1,C1,0.8866,666.4,343.33,0.008,16.043,0.0988
2,C2,0.0492,706.5,549.92,0.098,30.070,0.0783
3,C3,0.0246,616.0,666.06,0.152,44.097,0.0727
4,nC4,0.0098,550.6,765.62,0.193,58.123,0.0703
5,nC5,0.0098,488.6,845.80,0.251,72.151,0.0675


In [4]:
#Import binary coefficients
d= pd.read_excel("../ASSIGNMENTS/binary.xlsx").to_numpy()
d

array([[0.    , 0.018 , 0.039 , 0.046 , 0.047 , 0.048 ],
       [0.018 , 0.    , 0.005 , 0.01  , 0.0145, 0.0182],
       [0.039 , 0.005 , 0.    , 0.0017, 0.0032, 0.0048],
       [0.046 , 0.01  , 0.0017, 0.    , 0.0012, 0.0024],
       [0.047 , 0.0145, 0.0032, 0.0012, 0.    , 0.0008],
       [0.048 , 0.0182, 0.0048, 0.0024, 0.0008, 0.    ]])

In [5]:
Pci = component_properties['Pci'] 
Tci = component_properties['Tci']

Pri= P/Pci
Tri= (T+459.67 )/Tci

print('Reduced Pressure,  Pri:\n', Pri)
print('Reduced Temperature, Tri:\n',Tri)

Reduced Pressure,  Pri:
 0    1.520990
1    1.125450
2    1.061571
3    1.217532
4    1.362150
5    1.534998
Name: Pci, dtype: float64
Reduced Temperature, Tri:
 0    1.712910
1    1.134972
2    0.708594
3    0.585037
4    0.508960
5    0.460712
Name: Tci, dtype: float64


In [6]:
wi = component_properties['wi'] #accentric factor

w_a = 0.45723553
w_b = 0.07779607

for i in wi:
    if i<0.49:
        mi = 0.3796+1.54226*wi-0.2699*wi**2
    else:
        mi = 0.379642+1.48503*wi-0.164423*wi**2+0.016667*wi**3
        
Ai =(w_a*(1+mi*(1-Tri**0.5))**2*(Pri/Tri**2)).replace(np.nan,0).to_numpy()
Bi = ((w_b*Pri)/Tri).to_numpy()
print('Ai:',Ai)
print('Bi:',Bi)

Ai: [0.17688642 0.37927951 1.13501871 2.12458098 3.41173111 5.09117878]
Bi: [0.06907951 0.07714339 0.1165492  0.16190289 0.20820877 0.2592007 ]


In [7]:
#Calculating A and B

c = component_properties['composition'].to_numpy()
n= len(c)

B =  np.sum(c * Bi)

A=0
for i in range (n):
    for j in range (n):
        A += (1-d[i,j])*(Ai[i]*Ai[j])**0.5*c[i]*c[j] 
        
print('A :', A)
print('B :', B)

A : 0.46468266556708404
B : 0.08407456358023657


In [33]:
#Calculating Z 

m1 = 1+2**0.5
m2 = 1-2**0.5
p=1
q=(m1+m2-1)*B-1
r=(A+m1*m2*B**2-(m1+m2)*B*(B+1))
s=-A*B-m1*m2*B**2*(B+1)

Z = np.roots([p, q, r, s])

print('For Pressure:',P , 'and', 'Temperature:',T )
print('                          A:', round(A,4))
print('                          B:', round(B,4))
print('                          Z:', Z)
for i in range(len(Z)):
    if np.isreal(Z[i]):
        print('Without imaginery values, Z:', np.real(Z[i]))
        
print('                      Z_max:', max(Z))
print('                      Z_min:', min(Z))

For Pressure: 750.0 and Temperature: -70.0
                          A: 0.4647
                          B: 0.0841
                          Z: [0.47630776+0.j         0.21980884+0.13273547j 0.21980884-0.13273547j]
Without imaginery values, Z: 0.4763077590654529
                      Z_max: (0.4763077590654529+0j)
                      Z_min: (0.21980883867715534-0.1327354714317108j)


In [ ]:
Z